# Importing Libraries 📕

In [19]:
import os 
from langchain.document_loaders import PyMuPDFLoader
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
import pymongo
import pandas as pd

# API-KEY Setup 🗝️

In [20]:
os.environ['OPENAI_API_KEY'] = ''

# Extracting Text from PDF 🛠️

In [28]:
pdf_path = os.path.join('Resume', '05.pdf')

loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

In [29]:
loader.load()

[Document(page_content='Tammy Jones\nSacramento, CA 12345 \nTammyJones@sampleemail.com \n(123)456-7890\nPROFILE\nExperienced and hard working accounting professional with a track record of success in helping clients balance \nbooks and identify discrepancies. Detail oriented with a proven success record in account reconciliation and saving \nmoney. Possesses solid organizational skills and a willingness to learn more about accounting.\nPROFESSIONAL EXPERIENCE\nAccounting Specialist\nXYZ Accountants, Sacramento, CA | January 2012 - Present\n• \nEnsure accuracy of all transactions with clients, \nvendors and employees\n• \nApply acquired accounting skills when looking for \nways to save the company money\n• \nSaved XYZ over $250K in lost revenue through bank \nerrors\n• \nAcquired problem-solving skills in the SAP \naccounting system when working closely with the \ncompany CEO\n• \nImprove client relationships by resolving late and \nmissed payments\n• \nWork closely with management and 

# Embeddings, ChromaDB and QnAWithPDF 😶‍🌫️

In [23]:
embeddings = OpenAIEmbeddings()
persist_dir = os.path.join('Resume', 'storage')

In [24]:
vectordb = Chroma.from_documents(
    documents = documents,
    embedding = embeddings,
    persist_directory = persist_dir
)
vectordb.persist()

# Using Model For Summary Generation Of The Resume 👷‍♂️

In [25]:
retriever = vectordb.as_retriever(search_kwargs = {"k" : 3})
llm = ChatOpenAI()
qa =  RetrievalQA.from_chain_type(llm = llm, chain_type = "stuff", retriever = retriever)

In [26]:
# ui = input("Enter Prompt")
# ui = "Given the resume attached, could you please provide a concise summary highlighting the individual's key skills, experiences, and accomplishments? \n\n{documents}"
ui = '[key skills, experience, accompolishments]-words_limit[50]'
# ui = 'clear chat buffer [new resume input will be added soon]'
query = f"###Prompt {ui}"
llm_response = qa(query)
summary = llm_response['result']

# Making Dataset 💀

In [27]:
id = 1
df = pd.DataFrame({'ID': [id], 'Summary': [summary]})
df.to_csv('resume_summaries.csv', index = False)